In [1]:
!nvidia-smi

Fri Jan 27 15:34:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   49C    P8    16W /  70W |      2MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [49]:
!python --version

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Python 3.8.13


In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [3]:
import torch 
torch.__version__

'1.13.1+cu117'

In [4]:
import transformers
transformers.__version__

'4.22.2'

# unoptimized onnx

In [5]:
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForSeq2SeqLM


## vanilla test

In [6]:
%%time
tokenizer = AutoTokenizer.from_pretrained("hf_models/query_classify_v2")

CPU times: user 418 ms, sys: 108 ms, total: 526 ms
Wall time: 524 ms


In [7]:
%%time
model_ort = ORTModelForSeq2SeqLM.from_pretrained("hf_models/query_classify_v2", from_transformers=False, provider='CUDAExecutionProvider')

2023-01-27 15:35:01.232839930 [W:onnxruntime:, session_state.cc:1030 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2023-01-27 15:35:01.232868717 [W:onnxruntime:, session_state.cc:1032 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
2023-01-27 15:35:05.453577378 [W:onnxruntime:, session_state.cc:1030 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2023-01-27 15:35:05.453596718 [W:onnxruntime:, session_state.cc:1032 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
2023-01-27 15:35:09.189474085 [W:onnxrun

CPU times: user 6.72 s, sys: 6.26 s, total: 13 s
Wall time: 12.9 s


In [8]:
model_ort.device

device(type='cuda', index=0)

In [9]:
%%time
inputs = tokenizer("Generate taxonomy for query: dildo", return_tensors="pt").to('cuda')

CPU times: user 6.87 ms, sys: 7.36 ms, total: 14.2 ms
Wall time: 17.5 ms


In [10]:
%%time
inputs2 = tokenizer("Generate taxonomy for query: women gucci", return_tensors="pt").to('cuda')

CPU times: user 1.89 ms, sys: 0 ns, total: 1.89 ms
Wall time: 1.2 ms


In [11]:
%%time
inputs3 = tokenizer("Generate taxonomy for query: baby milk", return_tensors="pt").to('cuda')

CPU times: user 1.16 ms, sys: 668 µs, total: 1.83 ms
Wall time: 1.18 ms


In [28]:
%%time
outputs_beam = model_ort.generate(**inputs, num_beams=3, num_return_sequences=3)

CPU times: user 165 ms, sys: 12 ms, total: 177 ms
Wall time: 174 ms


/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [29]:
%%time
outputs = model_ort.generate(**inputs)

CPU times: user 115 ms, sys: 20.3 ms, total: 135 ms
Wall time: 133 ms


In [30]:
%%time
outputs_beam2 = model_ort.generate(**inputs2, num_beams=3, num_return_sequences=3)

CPU times: user 172 ms, sys: 28 ms, total: 200 ms
Wall time: 198 ms


In [31]:
%%time
outputs2 = model_ort.generate(**inputs2)

CPU times: user 82.7 ms, sys: 15.6 ms, total: 98.3 ms
Wall time: 96.4 ms


In [32]:
%%time
outputs_beam3 = model_ort.generate(**inputs3, num_beams=3, num_return_sequences=3)

CPU times: user 166 ms, sys: 25.1 ms, total: 191 ms
Wall time: 189 ms


/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [33]:
%%time
outputs3 = model_ort.generate(**inputs3)

CPU times: user 142 ms, sys: 12.3 ms, total: 155 ms
Wall time: 151 ms


/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [34]:
%%time
tokenizer.batch_decode(outputs)

CPU times: user 0 ns, sys: 1.57 ms, total: 1.57 ms
Wall time: 1.02 ms


['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']

In [35]:
%%time
tokenizer.batch_decode(outputs2)

CPU times: user 1.29 ms, sys: 68 µs, total: 1.36 ms
Wall time: 754 µs


["<pad> luggage & bags > women's bags</s>"]

In [36]:
%%time
tokenizer.batch_decode(outputs3)

CPU times: user 1.34 ms, sys: 207 µs, total: 1.55 ms
Wall time: 884 µs


['<pad> mother & kids > feeding > bottle feeding > baby food storage</s>']

In [37]:
%%time
tokenizer.batch_decode(outputs_beam)

CPU times: user 1.83 ms, sys: 284 µs, total: 2.11 ms
Wall time: 1.32 ms


['<pad> beauty & health > sexual wellness > sex toys > dildos</s>',
 '<pad> beauty & health > sexual wellness > sex toys</s><pad><pad><pad>',
 '<pad> beauty & health > sexual wellness > sm products > masturbation machine</s>']

In [38]:
%%time
tokenizer.batch_decode(outputs_beam2)

CPU times: user 1.48 ms, sys: 0 ns, total: 1.48 ms
Wall time: 1.54 ms


["<pad> luggage & bags > women's bags</s><pad><pad><pad><pad><pad>",
 "<pad> luggage & bags > women's bags > shoulder bags</s><pad>",
 "<pad> luggage & bags > women's bags > top-handle bags</s>"]

In [39]:
%%time
tokenizer.batch_decode(outputs_beam3)

CPU times: user 1.51 ms, sys: 193 µs, total: 1.7 ms
Wall time: 1.04 ms


['<pad> mother & kids > feeding > bottle feeding > baby food storage</s><pad>',
 '<pad> mother & kids > feeding > solid feeding > cups</s><pad><pad><pad>',
 '<pad> mother & kids > feeding > bottle feeding > baby bottles</s>']

## repeat test

In [40]:
%%timeit
tokenizer("Generate taxonomy for query: dildo", return_tensors="pt").to('cuda')

124 µs ± 2.41 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [41]:
%%timeit
tokenizer("Generate taxonomy for query: women gucci", return_tensors="pt").to('cuda')

125 µs ± 2.78 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [42]:
%%timeit
tokenizer("Generate taxonomy for query: baby milk", return_tensors="pt").to('cuda')

123 µs ± 965 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [43]:
%%timeit
model_ort.generate(**inputs, num_beams=3, num_return_sequences=3)

/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


166 ms ± 1.03 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [44]:
%%timeit
model_ort.generate(**inputs2, num_beams=3, num_return_sequences=3)

168 ms ± 1.32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [45]:
%%timeit
model_ort.generate(**inputs3, num_beams=3, num_return_sequences=3)

192 ms ± 2.62 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [46]:
%%timeit
model_ort.generate(**inputs)

130 ms ± 1.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [47]:
%%timeit
model_ort.generate(**inputs2)

92.6 ms ± 414 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [48]:
%%timeit
model_ort.generate(**inputs3)

148 ms ± 2.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [50]:
%%timeit
tokenizer.batch_decode(outputs)

32.5 µs ± 3.12 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [51]:
%%timeit
tokenizer.batch_decode(outputs2)

31.7 µs ± 2 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [52]:
%%timeit
tokenizer.batch_decode(outputs3)

32.1 µs ± 1.17 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [53]:
%%timeit
tokenizer.batch_decode(outputs_beam)

82.2 µs ± 953 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [54]:
%%timeit
tokenizer.batch_decode(outputs_beam2)

81.9 µs ± 1.26 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [55]:
%%timeit
tokenizer.batch_decode(outputs_beam3)

85.1 µs ± 1.37 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## all combined test

In [56]:
%%time
inputs = tokenizer("Generate taxonomy for query: dildo", return_tensors="pt").to('cuda')
outputs = model_ort.generate(**inputs)
tokenizer.batch_decode(outputs)

CPU times: user 131 ms, sys: 4.01 ms, total: 135 ms
Wall time: 133 ms


/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']

In [57]:
%%time
inputs2 = tokenizer("Generate taxonomy for query: women gucci", return_tensors="pt").to('cuda')
outputs2 = model_ort.generate(**inputs2)
tokenizer.batch_decode(outputs2)

CPU times: user 101 ms, sys: 11 µs, total: 101 ms
Wall time: 99.2 ms


/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


["<pad> luggage & bags > women's bags</s>"]

In [58]:
%%time
inputs3 = tokenizer("Generate taxonomy for query: baby milk", return_tensors="pt").to('cuda')
outputs3 = model_ort.generate(**inputs3)
tokenizer.batch_decode(outputs3)

CPU times: user 129 ms, sys: 28.2 ms, total: 157 ms
Wall time: 155 ms


['<pad> mother & kids > feeding > bottle feeding > baby food storage</s>']

In [59]:
%%time
inputs = tokenizer("Generate taxonomy for query: dildo", return_tensors="pt").to('cuda')
outputs_beam = model_ort.generate(**inputs, num_beams=3, num_return_sequences=3)
tokenizer.batch_decode(outputs_beam)

CPU times: user 151 ms, sys: 24.4 ms, total: 175 ms
Wall time: 173 ms


/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['<pad> beauty & health > sexual wellness > sex toys > dildos</s>',
 '<pad> beauty & health > sexual wellness > sex toys</s><pad><pad><pad>',
 '<pad> beauty & health > sexual wellness > sm products > masturbation machine</s>']

In [60]:
%%time
inputs2 = tokenizer("Generate taxonomy for query: women gucci", return_tensors="pt").to('cuda')
outputs_beam2 = model_ort.generate(**inputs2, num_beams=3, num_return_sequences=3)
tokenizer.batch_decode(outputs_beam2)

CPU times: user 164 ms, sys: 12 ms, total: 176 ms
Wall time: 174 ms


["<pad> luggage & bags > women's bags</s><pad><pad><pad><pad><pad>",
 "<pad> luggage & bags > women's bags > shoulder bags</s><pad>",
 "<pad> luggage & bags > women's bags > top-handle bags</s>"]

In [61]:
%%time
inputs3 = tokenizer("Generate taxonomy for query: baby milk", return_tensors="pt").to('cuda')
outputs_beam3 = model_ort.generate(**inputs3, num_beams=3, num_return_sequences=3)
tokenizer.batch_decode(outputs_beam3)

CPU times: user 192 ms, sys: 19.5 ms, total: 212 ms
Wall time: 209 ms


['<pad> mother & kids > feeding > bottle feeding > baby food storage</s><pad>',
 '<pad> mother & kids > feeding > solid feeding > cups</s><pad><pad><pad>',
 '<pad> mother & kids > feeding > bottle feeding > baby bottles</s>']

## repeat all combined test

In [62]:
%%timeit
inputs = tokenizer("Generate taxonomy for query: dildo", return_tensors="pt").to('cuda')
outputs = model_ort.generate(**inputs)
tokenizer.batch_decode(outputs)

/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


135 ms ± 1.09 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [63]:
%%timeit
inputs2 = tokenizer("Generate taxonomy for query: women gucci", return_tensors="pt").to('cuda')
outputs2 = model_ort.generate(**inputs2)
tokenizer.batch_decode(outputs2)

/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


96.8 ms ± 908 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [64]:
%%timeit
inputs3 = tokenizer("Generate taxonomy for query: baby milk", return_tensors="pt").to('cuda')
outputs3 = model_ort.generate(**inputs3)
tokenizer.batch_decode(outputs3)

154 ms ± 2.44 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [65]:
%%timeit
inputs = tokenizer("Generate taxonomy for query: dildo", return_tensors="pt").to('cuda')
outputs_beam = model_ort.generate(**inputs, num_beams=3, num_return_sequences=3)
tokenizer.batch_decode(outputs_beam)

/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


172 ms ± 2.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [66]:
%%timeit
inputs2 = tokenizer("Generate taxonomy for query: women gucci", return_tensors="pt").to('cuda')
outputs_beam2 = model_ort.generate(**inputs2, num_beams=3, num_return_sequences=3)
tokenizer.batch_decode(outputs_beam2)

171 ms ± 2.65 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [67]:
%%timeit
inputs3 = tokenizer("Generate taxonomy for query: baby milk", return_tensors="pt").to('cuda')
outputs_beam3 = model_ort.generate(**inputs3, num_beams=3, num_return_sequences=3)
tokenizer.batch_decode(outputs_beam3)

194 ms ± 2.88 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# compare with optimized onnx (no real difference, optimized onnx is actually slower)

In [7]:
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForSeq2SeqLM


tokenizer = AutoTokenizer.from_pretrained("hf_models/query_classify_v2")
model_ort = ORTModelForSeq2SeqLM.from_pretrained("hf_models/query_classify_v2_optimized", from_transformers=False, provider='CUDAExecutionProvider')


2023-01-26 03:02:45.213768679 [W:onnxruntime:, session_state.cc:1030 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2023-01-26 03:02:45.213798293 [W:onnxruntime:, session_state.cc:1032 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
2023-01-26 03:03:00.441794726 [W:onnxruntime:, session_state.cc:1030 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2023-01-26 03:03:00.441829182 [W:onnxruntime:, session_state.cc:1032 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
2023-01-26 03:03:15.654116970 [W:onnxrun

In [8]:
model_ort.device

device(type='cuda', index=0)

In [9]:
inputs = tokenizer("Generate taxonomy for query: dildo", return_tensors="pt").to('cuda')

In [10]:
%%timeit 
outputs = model_ort.generate(**inputs, num_beams=3, num_return_sequences=3)

183 ms ± 978 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%%timeit
outputs = model_ort.generate(**inputs)

138 ms ± 1.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# compare with pytorch

In [12]:
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoModel,
    AutoTokenizer
)

In [13]:
model_pt = AutoModelForSeq2SeqLM.from_pretrained("hf_models/query_classify_v2")

In [14]:
tokenizer = AutoTokenizer.from_pretrained("hf_models/query_classify_v2")

In [15]:
model_pt.eval()
model_pt.cuda()
tmp = 2

In [16]:
inputs = tokenizer("Generate taxonomy for query: dildo", return_tensors="pt").to('cuda')

## vanilla - ONNX is (333-181)/333 = 45% faster for beam-search, (241-134) / 241 = 44% faster for greedy-decoding

In [17]:
%%timeit
outputs = model_pt.generate(**inputs, num_beams=3, num_return_sequences=3)

333 ms ± 10.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%timeit
outputs = model_pt.generate(**inputs)

241 ms ± 4.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## inference mode - ONNX is (306-181)/306 = 41% faster for beam-search, (219-134)/219 = 39% faster for greedy-decoding

In [20]:
import torch

In [21]:
%%timeit
with torch.inference_mode():
    outputs = model_pt.generate(**inputs, num_beams=3, num_return_sequences=3)

/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


306 ms ± 2.37 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%%timeit
with torch.inference_mode():
    outputs = model_pt.generate(**inputs)

219 ms ± 1.75 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## fp16 autocast inference mode (actually slower than without fp16)

In [23]:
%%timeit
with torch.inference_mode(), torch.autocast(dtype=torch.float16, cache_enabled=True, device_type="cuda"):
    outputs = model_pt.generate(**inputs, num_beams=3, num_return_sequences=3)

/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


314 ms ± 2.54 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
%%timeit
with torch.inference_mode(), torch.autocast(dtype=torch.float16, cache_enabled=True, device_type="cuda"):
    outputs = model_pt.generate(**inputs)

238 ms ± 3.65 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
